In [ ]:
!unzip /content/ck-aio-hutech (1).zip -d /content/

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `unzip /content/ck-aio-hutech (1).zip -d /content/'


In [ ]:
### START: CÁC KHAI BÁO CHÍNH - KHÔNG THAY ĐỔI ###
SEED = 46  # Số seed (Ban tổ chức sẽ công bố & thay đổi vào lúc chấm)
TRAIN_DATA_DIR_PATH = '/content/train2'
TEST_DATA_DIR_PATH = '/content/train2/tuyết khô'
PATH_OAI086 = "/content"
Submission_path = "/content/sample_data"
### END: CÁC KHAI BÁO CHÍNH - KHÔNG THAY ĐỔI ###

### START: CÁC THƯ VIỆN IMPORT ###
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.io import read_image
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import StepLR
from torchvision.models import ResNet101_Weights
import os
import re
import numpy as np
import pandas as pd
import unicodedata
import gc
import torchvision.transforms.functional as TF
import random
torch.cuda.empty_cache()
gc.collect()
### END: CÁC THƯ VIỆN IMPORT ###

### START: SEEDING EVERYTHING - KHÔNG THAY ĐỔI ###
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
### END: SEEDING EVERYTHING - KHÔNG THAY ĐỔI ###

### START: ĐỊNH NGHĨA & CHẠY HUẤN LUYỆN MÔ HÌNH ###
IMG_SIZE = (224, 224)
BATCH_SIZE = 10
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device with {torch.cuda.device_count()} GPUs")

# Hàm chuẩn hóa tên thư mục
def normalize_folder_name(name):
    name = ''.join(c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    name = name.replace('+', ' ').strip()
    return name.lower()

# Transform cho Grayscale
train_transform_grayscale = transforms.Compose([
    transforms.Lambda(lambda x: TF.rgb_to_grayscale(x)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
    transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
    transforms.Lambda(lambda x: x / 255.0),
])

test_transform_grayscale = transforms.Compose([
    transforms.Lambda(lambda x: TF.rgb_to_grayscale(x)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
    transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
    transforms.Lambda(lambda x: x / 255.0),
])

# Transform cho RGB
train_transform_rgb = transforms.Compose([
    transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
    transforms.Lambda(lambda x: x / 255.0),
])

test_transform_rgb = transforms.Compose([
    transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
    transforms.Lambda(lambda x: x / 255.0),
])

# Dataset tùy chỉnh cho tập huấn luyện
class CustomTrainDataset(Dataset):
    def __init__(self, root, transform=None, augmentations=None, augment_classes=None, include_classes=None):
        self.root = root
        self.transform = transform
        self.classes = sorted(os.listdir(root))
        print("Danh sách thư mục thực tế:", self.classes)

        # Định nghĩa ánh xạ lớp
        self.class_to_idx = {
            normalize_folder_name('dong co'): 0,
            normalize_folder_name('tai meo'): 1,
            normalize_folder_name('tuyet kho'): 2,
        }
        print("Class to index mapping:", self.class_to_idx)
        # Định nghĩa ánh xạ lớp
        self.class_to_idx = {
            normalize_folder_name('dong co'): 0,
            normalize_folder_name('tai meo'): 1,
            normalize_folder_name('tuyet kho'): 2,
            normalize_folder_name('đông cô'): 0, # Added mapping for 'đông cô'
        }
        # Kiểm tra xem các lớp thực tế có khớp với class_to_idx không
        for cls in self.classes:
            normalized_cls = normalize_folder_name(cls)
            if normalized_cls not in self.class_to_idx:
                raise ValueError(f"Thư mục '{cls}' không được ánh xạ trong class_to_idx.")

        self.augment_class_indices = (
            [self.class_to_idx[cls] for cls in augment_classes]
            if augment_classes is not None else []
        )
        self.include_classes = include_classes if include_classes is not None else list(self.class_to_idx.values())
        self.images = []
        self.labels = []

        for cls in self.classes:
            cls_idx = self.class_to_idx[normalize_folder_name(cls)]
            if cls_idx in self.include_classes:
                cls_path = os.path.join(root, cls)
                for img_name in os.listdir(cls_path):
                    if img_name.endswith(('.png', '.jpg', '.jpeg')):
                        self.images.append(os.path.join(cls_path, img_name))
                        # Ánh xạ nhãn cho Grayscale hoặc RGB
                        if self.include_classes == [0, 1]:
                            self.labels.append(0 if cls_idx == 0 else 1)
                        else:
                            self.labels.append(cls_idx)
        print("Labels created:", set(self.labels))

        if not self.images:
            raise ValueError("Không tìm thấy hình ảnh nào trong các thư mục được chỉ định.")

    def __len__(self):
        total = 0
        for label in self.labels:
            if label in self.augment_class_indices:
                total += 1 + (len(self.augmentations) if self.augmentations else 0)
            else:
                total += 1
        return total

    def __getitem__(self, idx):
        current_idx = 0
        for i, label in enumerate(self.labels):
            num_versions = 1 + (len(self.augmentations) if label in self.augment_class_indices and self.augmentations else 0)
            if idx < current_idx + num_versions:
                img_idx = i
                aug_idx = idx - current_idx
                break
            current_idx += num_versions

        img_path = self.images[img_idx]
        label = self.labels[img_idx]
        image = read_image(img_path).float()

        if label in self.augment_class_indices and self.augmentations and aug_idx > 0:
            aug_transform = self.augmentations[aug_idx - 1]
            image = aug_transform(image)
        if self.transform:
            image = self.transform(image)

        return image, label

# Dataset cho tập test
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted(
            [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))],
            key=lambda x: int(re.sub('\D', '', x))
        )

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])
        image = read_image(img_path).float()
        file_name = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image, file_name

# Augmentations với grayscale
augmentations_grayscale = [
    transforms.Compose([
        transforms.Lambda(lambda x: TF.rgb_to_grayscale(x)),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
        transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
        transforms.RandomHorizontalFlip(p=1),
        transforms.Lambda(lambda x: x / 255.0),
    ]),
    transforms.Compose([
        transforms.Lambda(lambda x: TF.rgb_to_grayscale(x)),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
        transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
        transforms.RandomVerticalFlip(p=1),
        transforms.Lambda(lambda x: x / 255.0),
    ]),
]

# Augmentations với RGB
augmentations_rgb = [
    transforms.Compose([
        transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
        transforms.RandomHorizontalFlip(p=1),
        transforms.Lambda(lambda x: x / 255.0),
    ]),
    transforms.Compose([
        transforms.Lambda(lambda x: F.interpolate(x.unsqueeze(0), size=IMG_SIZE, mode='bilinear', align_corners=False).squeeze(0)),
        transforms.RandomVerticalFlip(p=1),
        transforms.Lambda(lambda x: x / 255.0),
    ]),
]

# Tạo dataset và dataloader
Train_ds_grayscale = CustomTrainDataset(
    root=TRAIN_DATA_DIR_PATH,
    transform=train_transform_grayscale,
    include_classes=[0, 1]
)
Train_loader_grayscale = DataLoader(Train_ds_grayscale, batch_size=BATCH_SIZE, shuffle=True)
Test_ds_grayscale = TestDataset(root_dir=TEST_DATA_DIR_PATH, transform=test_transform_grayscale)
Test_loader_grayscale = DataLoader(Test_ds_grayscale, batch_size=BATCH_SIZE, shuffle=False)

Train_ds_rgb = CustomTrainDataset(
    root=TRAIN_DATA_DIR_PATH,
    transform=train_transform_rgb,
    include_classes=[0, 1, 2]
)
Train_loader_rgb = DataLoader(Train_ds_rgb, batch_size=BATCH_SIZE, shuffle=True)
Test_ds_rgb = TestDataset(root_dir=TEST_DATA_DIR_PATH, transform=test_transform_rgb)
Test_loader_rgb = DataLoader(Test_ds_rgb, batch_size=BATCH_SIZE, shuffle=False)

test_filename = [file_name for _, file_name in Test_ds_grayscale]

print(f"Số mẫu trong Train_ds_grayscale (lớp 0,1): {len(Train_ds_grayscale)}")
print(f"Số lớp trong Train_ds_grayscale: {len(set(Train_ds_grayscale.labels))}")
print(f"Số mẫu trong Train_ds_rgb (lớp 0,1,2): {len(Train_ds_rgb)}")
print(f"Số lớp trong Train_ds_rgb: {len(set(Train_ds_rgb.labels))}")

# Các lớp mô hình CBAM và ResNetCBAM
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
        )
        self.pool_types = pool_types

    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type == 'avg':
                avg_pool = F.avg_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp(avg_pool)
            elif pool_type == 'max':
                max_pool = F.max_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp(max_pool)
            elif pool_type == 'lp':
                lp_pool = F.lp_pool2d(x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp(lp_pool)
            elif pool_type == 'lse':
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp(lse_pool)

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid(channel_att_sum).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), torch.mean(x, 1).unsqueeze(1)), dim=1)

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)

    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out)
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False, no_channel=False):
        super(CBAM, self).__init__()
        self.no_channel = no_channel
        if not no_channel:
            self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial = no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x_out = x
        if not self.no_channel:
            x_out = self.ChannelGate(x_out)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out

class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=4, weights=ResNet101_Weights.IMAGENET1K_V1, no_channel=False):
        super(ResNetCBAM, self).__init__()
        self.resnet = models.resnet101(weights=weights)
        self.cbam1 = CBAM(256, no_channel=no_channel)
        self.cbam2 = CBAM(512, no_channel=no_channel)
        self.cbam3 = CBAM(1024, no_channel=no_channel)
        self.cbam4 = CBAM(2048, no_channel=no_channel)
        self.dropout = nn.Dropout(0.5)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.cbam1(x)
        x = self.resnet.layer2(x)
        x = self.cbam2(x)
        x = self.resnet.layer3(x)
        x = self.cbam3(x)
        x = self.resnet.layer4(x)
        x = self.cbam4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.resnet.fc(x)
        return x

class ResNetCBAM2(nn.Module):
    def __init__(self, num_classes=4, weights=ResNet101_Weights.IMAGENET1K_V1):
        super(ResNetCBAM2, self).__init__()
        self.resnet = models.resnet101(weights=weights)
        self.cbam1 = CBAM(256)
        self.cbam2 = CBAM(512)
        self.cbam3 = CBAM(1024)
        self.cbam4 = CBAM(2048)
        self.dropout = nn.Dropout(0.5)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.cbam1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.cbam4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.resnet.fc(x)
        return x

# Hàm mất mát LabelSmoothingLoss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

# Hàm mất mát FocalLoss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

# Hàm huấn luyện
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    total_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / size
    print(f"Train Loss: {avg_loss:.6f}, Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

# Hàm test với xác suất
def test_with_probs(dataloader, model, is_grayscale):
    model.eval()
    predictions = []
    probabilities = []

    with torch.no_grad():
        for X, _ in dataloader:
            X = X.to(device)
            pred = model(X)
            probs = F.softmax(pred, dim=1)
            pred_classes = pred.argmax(1)
            predictions.extend(pred_classes.cpu().numpy())
            probabilities.extend(probs.cpu().numpy())

    return predictions, probabilities

# Giai đoạn 1: Mô hình Grayscale (chỉ lớp 0,1, num_classes=2)
print("Phase 1:")
model_grayscale = ResNetCBAM(num_classes=2, no_channel=True)
for param in model_grayscale.parameters():
    param.requires_grad = False
for param in model_grayscale.resnet.layer2.parameters():
    param.requires_grad = True
for param in model_grayscale.resnet.layer3.parameters():
    param.requires_grad = True
for param in model_grayscale.resnet.layer4.parameters():
    param.requires_grad = True
for param in model_grayscale.resnet.fc.parameters():
    param.requires_grad = True

model_grayscale = model_grayscale.to(device)

optimizer_grayscale = torch.optim.Adam(filter(lambda p: p.requires_grad, model_grayscale.parameters()), lr=0.00005)
class_weights_grayscale = torch.tensor([1.0, 1.0], dtype=torch.float).to(device)
loss_fn_grayscale = LabelSmoothingLoss(classes=2, smoothing=0.05)
scheduler_grayscale = torch.optim.lr_scheduler.StepLR(optimizer_grayscale, step_size=5, gamma=0.5)

epochs_grayscale = 15
for t in range(epochs_grayscale):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_accuracy = train(Train_loader_grayscale, model_grayscale, loss_fn_grayscale, optimizer_grayscale)
    if t < epochs_grayscale - 1:
        scheduler_grayscale.step()

# Giai đoạn 2: Mô hình RGB (tất cả lớp 0,1,2, num_classes=3)
print("\nPhase 2:")
model_rgb = ResNetCBAM2(num_classes=3)
for param in model_rgb.parameters():
    param.requires_grad = False
for param in model_rgb.resnet.layer2.parameters():
    param.requires_grad = True
for param in model_rgb.resnet.layer3.parameters():
    param.requires_grad = True
for param in model_rgb.resnet.layer4.parameters():
    param.requires_grad = True
for param in model_rgb.resnet.fc.parameters():
    param.requires_grad = True

model_rgb = model_rgb.to(device)

optimizer_rgb = torch.optim.Adam(filter(lambda p: p.requires_grad, model_rgb.parameters()), lr=0.00005)
class_weights_rgb = torch.tensor([1.0, 1.0, 2.5], dtype=torch.float).to(device)
loss_fn_rgb = FocalLoss(gamma=2.5, alpha=class_weights_rgb)
scheduler_rgb = torch.optim.lr_scheduler.StepLR(optimizer_rgb, step_size=5, gamma=0.5)

epochs_rgb = 10
for t in range(epochs_rgb):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_accuracy = train(Train_loader_rgb, model_rgb, loss_fn_rgb, optimizer_rgb)
    if t < epochs_rgb - 1:
        scheduler_rgb.step()

# Thực hiện test với xác suất
pred_grayscale, probs_grayscale = test_with_probs(Test_loader_grayscale, model_grayscale, is_grayscale=True)
pred_rgb, probs_rgb = test_with_probs(Test_loader_rgb, model_rgb, is_grayscale=False)

# Kết hợp dự đoán dựa trên xác suất
print("\nCombining Predictions...")
final_predictions = []
for i in range(len(pred_rgb)):
    if pred_rgb[i] == 1 and probs_rgb[i][1] > 0.7:  # Ưu tiên RGB cho nhãn 1 nếu xác suất cao
        final_predictions.append(pred_rgb[i])
    else:
        # Sử dụng Grayscale cho nhãn 0 hoặc 1, nếu không thì lấy RGB
        if pred_grayscale[i] in [0, 1]:
            final_predictions.append(pred_grayscale[i])
        else:
            final_predictions.append(pred_rgb[i])

# Lưu kết quả cuối cùng
results_df = pd.DataFrame({'image_name': test_filename, 'label': final_predictions})
results_df.to_csv(os.path.join(Submission_path, 'results.csv'), index=False)
print("FINISH")

Using cuda device with 1 GPUs
Danh sách thư mục thực tế: ['tai mèo', 'tuyết khô', 'đông cô']
Class to index mapping: {'dong co': 0, 'tai meo': 1, 'tuyet kho': 2}
Labels created: {0, 1}
Danh sách thư mục thực tế: ['tai mèo', 'tuyết khô', 'đông cô']
Class to index mapping: {'dong co': 0, 'tai meo': 1, 'tuyet kho': 2}
Labels created: {0, 1, 2}
Số mẫu trong Train_ds_grayscale (lớp 0,1): 700
Số lớp trong Train_ds_grayscale: 2
Số mẫu trong Train_ds_rgb (lớp 0,1,2): 1050
Số lớp trong Train_ds_rgb: 3
Phase 1:


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 188MB/s]


Epoch 1
-------------------------------
Train Loss: 0.345567, Accuracy: 0.9271
Epoch 2
-------------------------------
Train Loss: 0.237279, Accuracy: 0.9914
Epoch 3
-------------------------------
Train Loss: 0.230538, Accuracy: 0.9943
Epoch 4
-------------------------------
Train Loss: 0.223587, Accuracy: 0.9971
Epoch 5
-------------------------------
Train Loss: 0.213211, Accuracy: 1.0000
Epoch 6
-------------------------------
Train Loss: 0.219199, Accuracy: 0.9929
Epoch 7
-------------------------------
Train Loss: 0.215023, Accuracy: 0.9986
Epoch 8
-------------------------------
Train Loss: 0.212551, Accuracy: 0.9986
Epoch 9
-------------------------------
Train Loss: 0.209702, Accuracy: 1.0000
Epoch 10
-------------------------------
Train Loss: 0.207108, Accuracy: 1.0000
Epoch 11
-------------------------------
Train Loss: 0.207865, Accuracy: 0.9986
Epoch 12
-------------------------------
Train Loss: 0.205863, Accuracy: 0.9986
Epoch 13
-------------------------------
Train Lo

In [ ]:
### END: THỰC NGHIỆM & XUẤT FILE KẾT QUẢ RA CSV ###
Submission_path = "/content/"


# Kết hợp dự đoán
print("\nCombining Predictions... ")
final_predictions = []
for i in range(len(pred_rgb)):
    if pred_rgb[i] == 1:
        final_predictions.append(pred_rgb[i])
    else:
        final_predictions.append(pred_grayscale[i])

# Lưu kết quả cuối cùng
results_df = pd.DataFrame({'image_name': test_filename, 'label': final_predictions})
results_df.to_csv(os.path.join(Submission_path,'results.csv'), index=False)
print("FINISH")


Combining Predictions... 
FINISH
